# Django Intro

Django is a high-level Python Web Framework that allows for maintainable websites.

From the previous lessons, we know that a Wep Application waits for HTTP requests from the Web Browser. The App gets the URl, and other data. It may need to read or write info from a database, or do other tasks to satisfy the request. The app will return a response to the Browser, and can dynamically create an HTML page for the Browser.

Django Web Apps groups the code to handle each of these steps into separate files:

<img src="basic-django.png">

The process starts by the Django Web App getting the HTTP request. It then goes into:
1. `URL (urls.py)`: It's possible to process every single URL request from a single function, but we instead write a separate view function to handle each resource. We use a URL mapper to redirect HTTP requests to the right view based on the request URL. The URL mapper can also pass the URL parameters/patterns to the view function as data.
2. `View (views.py)`: A view is a request handler function. It gets HTTP requests and returns HTTP responses. Views get the data to satisfy requests using **models**, and delegate formatting of the response to the templates.
3. `Model (models.py)`: A model defines the application's data structure, and lets us manage, add, modify, and query records in the database.
4. `Template (<filename>.html)`: Text file defining the structure or layout of a file, with placeholders. It's the view that creates an HTML page using the HTML template, and populates it with data from a model. Note, that it doesn't have to be HTML.

Django uses this organization as the "Model View Template (MVT)" architecture. Similar to the MVC architecture.

We will now take a quick look at what these 4 main parts look like, but we will go into more detail later on.

## 1. Sending the request to the right view (`urls.py`)

In `urls.py`, we will store the URL mapper. Here is the mapper:

In [ ]:
urlpatterns = [
    path('admin/', admin.site.urls),
    path('book/<int:id>/', views.book_detail, name='book_detail'),
    path('catalog/', include('catalog.urls')),
    re_path(r'^([0-9]+)/$', views.best),
]

Ignore the errors for now.  
`urlpatterns` IS the mapper, and it defines a list of mappings between ROUTES and their corresponding view function. Routes are URL patterns. If an HTTP request is recieved and has a URL matching a specified pattern, then the associated view function will be called and passed with the request.

`urlpatterns`'s elements are `path` and/or `re_path` functions.  
The first argument in both of the functions is a route/URL pattern that will be matched. The angle brackets, `<>`, in a URL will be passed to the view function as arguments. The `re_path` function uses a regular expression, which we will learn later.  
The second argument is another function that will be called when the pattern is matched.

## 2. Handling the request (`views.py`)

Views are the HEART of the Web App. It gets HTTP requests and sends HTTP responses. In between the request/response, it accesses a database, render templates, etc.

Let's look at a view:

In [ ]:
# filename: views.py (Django view functions)

from django.http import HttpResponse

def index(request):
    # Get an HttpRequest - the request parameter
    # perform operations using information from the request.
    # Return HttpResponse
    return HttpResponse('Hello from Django!')


`index()` is a view function, which the URL mapper before in `urls.py` could have called. It receives the HTTP request as a parameter, and returns an HTTP response. In this case, nothing happens with the request, and we return a hard coded string. Later, we will mess with the request.

## 3. Defining data models (`models.py`)

We use models to manage and query data through Python. Models define the structure of the data, using field types and their maximum size, default values, selection list options, help text, label text for forms, etc. Once we choose a database we want to use, we don't need to talk to it directly at all. We just write the model structure and other code, and Django handles the dirty work of communicating with the database for us.

Here is an example:

In [ ]:
# filename: models.py

from django.db import models

class Team(models.Model):
    team_name = models.CharField(max_length=40)

    TEAM_LEVELS = (
        ('U09', 'Under 09s'),
        ('U10', 'Under 10s'),
        ('U11', 'Under 11s'),
        # …
        # list other team levels
    )
    team_level = models.CharField(max_length=3, choices=TEAM_LEVELS, default='U11')

This `Team` class is a `Model`. It defines `team_name` and `team_level` as character fields for us. It also sets characteristics for each one. We also specify `team_level` as a choice field by using the `choices` parameter. 

## 3.5. Querying Data (`views.py`)

We can also search the associated database using Django's model and view. Here is an example of querying:

In [ ]:
## filename: views.py

from django.shortcuts import render
from .models import Team

def index(request):
    list_teams = Team.objects.filter(team_level__exact="U09")
    context = {'youngest_teams': list_teams}
    return render(request, '/best/index.html', context)


So instead of having an empty view like before, now we take the `Team` object that we created, and get all records where `team_level` is EXACTLY `"U09"`.

It also uses the `render` function to create the HttpResponse that is sent back to the browser. This function is a SHORTCUT, as it creates an HTML file by combining a specified HTML template, and some data to insert into the template (data being `context`).

## 4. Rendering data (HTML Templates)

Template systems allows us to structure an output document, while using placeholders for data that will be filled in when the page is generated. Django can support multiple template systems.

Here is an example:

In [ ]:
## filename: best/templates/best/index.html

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Home page</title>
</head>
<body>
  {% if youngest_teams %}
    <ul>
      {% for team in youngest_teams %}
        <li>{{ team.team_name }}</li>
      {% endfor %}
    </ul>
  {% else %}
    <p>No teams are available.</p>
  {% endif %}
</body>
</html>

The double handle bars are placeholders. We also use `{% expression %}` notation to allow for expressions like conditional rendering, and loop rendering. 

# Other Django capabilities

In almost every single web application, we will need to use the URL mapping, views, models, and templates. But there are other things provided by Django:

* **Forms**: HTML forms allow use to collect user data. Django simplifies form creation, validation, and processing.
* **User Authentication and Permissions**: Django includes these things with security in mind.
* **Caching**: Django provides flexible caching to store a part or all of a rendered page so it doesn't get re-rendered except when necessary.
* **Administration Site**: This is included by default when creating an app using the basic skeleton. Its here to easily create, edit, and view and data models in the site.
* **Serializing Data**: Django makes it easy to serve data as XML or JSON. This is useful when creating a web service, or when creating a website in which the client-side code handles all the rendering of data.

# Setting Up a Dev Environment

In this tutorial, we will use `SQLite`, which stores its data in a file. It is lightweight, and is good for primarily "read-only" applications.

We can either install system-wide or in a Python virtual environment. The reason it matters is because if we install Django in a global environment, we will only be able to target one version of Django on the computer. This is a problem when creating new websites using the newest version of Django, while still maintaining websites that rely on older versions. Django dev team recommends we use Python virtual environments.

For windows, we will need to install `virtualenvwrapper-win` by using:

In [ ]:
pip install virtualenvwrapper-win

For macos, we will need to install `virtualenvwrapper` by doing:

In [ ]:
sudo pip3 install virtualenvwrapper

### Note, only macos will need to do the steps below

Then we need to add the following lines to the end of the shell startup file:

In [ ]:
export WORKON_HOME=$HOME/.virtualenvs
export VIRTUALENVWRAPPER_PYTHON=/usr/bin/python3
export PROJECT_HOME=$HOME/Devel
source /usr/local/bin/virtualenvwrapper.sh

Look at the `VIRTUALENVWRAPPER_PYTHON=/usr/...python3`, you must change this if Python and the script are not in the expected location. You can use the commands `which virtualenvwrapper.sh` and `which python3` to find the locations for your system.

Reload the startup file by doing:

In [1]:
source ~/.bash_profile

SyntaxError: invalid syntax (2246048675.py, line 1)

### We should be able to create a new virtual environment with the `mkvirtualenv name_of_virtualenv` command. (On all systems)

After running this command, some text will pop up

Once in, we will be using these commands mainly:

* `deactivate` - Exit out of current Python virtual env
* `workon` - List available virtual env
* `workon name_of_environment` - Activate the specified Python virtual env
* `rmvirtualevn name_of_environment` - Remove the specified evn

We should call the `workon name_of_environment` command to enter the virtual env, so we can use pip

Now we install Django using:

In [ ]:
pip3 install django~=4.0

Now we test it:

In [2]:
# Linux/macOS
python3 -m django --version
 4.0.2

# Windows
py -3 -m django --version
 4.0.2

# or on Windows
py -m django --version

SyntaxError: invalid syntax (1613713844.py, line 2)

Now let's call:

In [ ]:
mkdir django_test
cd django_test

Then:

In [ ]:
django-admin startproject mytestsite
cd mytestsite

And finally, we will run the development web server from within this folder using `manage.py` and the `runserver` command:

In [ ]:
python manage.py runserver

We will get errors, but that is fine

Go to the url it tells us to go to. and it should work!